#AutoEncoders

##Downloading the dataset

###ML-100K

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2022-06-05 13:36:07--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  29.4MB/s    in 0.2s    

2022-06-05 13:36:07 (29.4 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2022-06-05 13:36:15--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  32.5MB/s    in 0.2s    

2022-06-05 13:36:16 (32.5 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:
# all movies.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# all users
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [23]:
# all ratings
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings.rename({0:"UserID", 1:"MovieID", 2:"Rating", 3:"Timestamp"}, axis= 1, inplace= True)
ratings["Date"] = pd.to_datetime(ratings["Timestamp"], unit='s').apply(lambda x : x.date())
ratings.head()

,UserID,MovieID,Rating,Timestamp,Date
0,1,1193,5,978300760,2000-12-31
1,1,661,3,978302109,2000-12-31
2,1,914,3,978301968,2000-12-31
3,1,3408,4,978300275,2000-12-31
4,1,2355,5,978824291,2001-01-06


## Preparing the training set and the test set


In [42]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int') # convert to numpy array to be used with pytorch
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int') # convert to numpy array to be used with pytorch

print("All ratings:\t", len(ratings), "\t", (len(ratings) / len(ratings) * 100), "%")
print("Training set:\t", len(training_set), "\t\t", (len(training_set) / len(ratings) * 100), "%")
print("Test set:\t", len(test_set), "\t\t", (len(test_set) / len(ratings) * 100), "%")

All ratings:	 1000209 	 100.0 %
Training set:	 79999 		 7.998228370270613 %
Test set:	 19999 		 1.9994821082393779 %


## Getting the number of users and movies


In [43]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [44]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [45]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [46]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [47]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7713)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0381)
epoch: 5loss: tensor(1.0310)
epoch: 6loss: tensor(1.0264)
epoch: 7loss: tensor(1.0240)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0209)
epoch: 10loss: tensor(1.0199)
epoch: 11loss: tensor(1.0189)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0176)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0169)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0166)
epoch: 19loss: tensor(1.0165)
epoch: 20loss: tensor(1.0164)
epoch: 21loss: tensor(1.0164)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0161)
epoch: 24loss: tensor(1.0157)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0157)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0152)
epoch: 29loss: tensor(1.0132)
epoch: 30loss: tensor(1.0122)
epoch: 31loss: tensor(1.0087)
epoch: 32loss: tensor(1.0097)
epoch: 33loss: tensor(1.0049)
epoch: 34loss: tens

## Testing the SAE


In [48]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9535)
